In [6]:
import warnings
warnings.filterwarnings(f"ignore")

import pandas as pd
from tqdm import tqdm

import shared
shared.init()
from Features.compute_curve import get_curve
from Features.compute_transfer_features import get_transfer_features

In [21]:
pd.read_csv("data/sushiswap/polygon_pools.csv")['token0'].tolist()+pd.read_csv("data/sushiswap/polygon_pools.csv")['token1'].tolist()

['...',
 ...]

In [12]:
hours_since_creation = 24
protocol = "sushiswap"

pools = pd.read_csv(f"./data/{protocol}/polygon_pools.csv")
pools = pools.loc[(pools.token0 == shared.WMATIC) | (pools.token1 == shared.WMATIC)]

tokens = pd.read_csv(f"./data/{protocol}/labelled_data.csv")
time_elapsed = 60*60*hours_since_creation

total_features = {}
total_transfers = pd.DataFrame(columns=['token', 'address', 'price_rev'])

In [13]:
tokens

,0x472DB1061B5C78014Dcdf744648494DcA8197DA7,0
0,0xAa9654BECca45B5BDFA5ac646c939C62b527D394,0
1,0x0775d7C64fcB6DA6Fd855620dC3cB881461676Ba,0
2,0xff30636B5B2d4497c65d08e8798D86987Fdc2F2E,0
3,0x361d59565DdCEa2e9C2AFe98bbF4298984E1cBbe,0
4,0x172370d5Cd63279eFa6d502DAB29171933a610AF,0
...,...,...
1248,0x5d5530eb3147152FE78d5C4bFEeDe054c8d1442A,1
1249,0x53f8036e43b2EF39cFCD0C5f26C1257a85D5A3B2,1
1250,0x76e63a3E7Ba1e2E61D3DA86a87479f983dE89a7E,1
1251,0x9085B4d52c3e0B8B6F9AF6213E85A433c7D76f19,1


In [16]:
for token_info in tqdm(tokens.values):
    token_address, label = token_info[0], token_info[1]
    pool = pools.loc[(pools.token0 == token_address) | (pools.token1 == token_address)].values[0]
    pool_address, token0, token1, block_number, pool_creation_timestamp = pool[0], pool[1], pool[2], pool[3], pool[4]
    total_features[pool_address] = {'token_address': token_address, 'label': label}

    position = 1 if token1 == shared.WMATIC else 0
    token = token0 if position == 1 else token1
    limit_timestamp = pool_creation_timestamp + time_elapsed

    transfers = pd.read_csv(f"./data/{protocol}/transfer/{token}.csv")
    mints = pd.read_csv(f"./data/{protocol}/mint/{pool_address}.csv")
    burns = pd.read_csv(f"./data/{protocol}/burn/{pool_address}.csv")
    swaps = pd.read_csv(f"./data/{protocol}/swap/{pool_address}.csv")

    transfers = transfers.loc[transfers.timestamps < limit_timestamp]
    mints = mints.loc[mints.timestamps < limit_timestamp]
    burns = burns.loc[burns.timestamps < limit_timestamp]
    swaps = swaps.loc[swaps.timestamps < limit_timestamp]

    total_features[pool_address].update({'num_mints': len(mints), 'num_burns': len(burns), 'num_swaps': len(swaps)})

    if 0 in [len(transfers)]:
        continue

    # Pool features
    total_supply = + sum(transfers.loc[(transfers['from'] == shared.ETH_ADDRESS)]['value'].astype(float))\
                   - sum(transfers.loc[(transfers['to'] == shared.ETH_ADDRESS)]['value'].astype(float))
    if total_supply == 0:
        continue

    # Transfer features
    tx_curve = get_curve(transfers.values, type_='tx')
    total_features[pool_address].update(tx_curve)
    transfer_features = get_transfer_features(transfers, total_supply)
    total_features[pool_address].update(transfer_features)
    total_features[pool_address].update({'difference_token_pool': + pool_creation_timestamp
                                                                  - transfers.iloc[0]['timestamps']})

pd.DataFrame(total_features).transpose().to_csv(f"./data/{protocol}/features{hours_since_creation}h.csv", index_label="pool_address")

  0%|          | 0/1253 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: './data/sushiswap/transfer/0xAa9654BECca45B5BDFA5ac646c939C62b527D394.csv'

In [3]:
tokens

,0
0,0xC6c1f684151eb235472139477F869b2e8385C416
1,0xa9B7F1f421294e8Aae4f0fE237af84dF0a747185
2,0xCD6EaF64087a1F3AE6ae5Ab4cdf83568a4152F9e
3,0xdf9B4b57865B403e08c85568442f95c26b7896b0
4,0xA798bA9A2188ad61Be6b930C5B6a69a3C7d4aed3
...,...
298,0x3B9b24991fC6F3CBe518843cBcd43fB937A65FB8
299,0xE516be297c846a4a4bF9ec163D6864BB552825Ef
300,0x041dEE4Be8Dd8A5C2927444A533e30320ffF50D3
301,0xa42fc534e7bc353Cd230EE399BC4c7141837FEEC


In [17]:
pool

array(['0x345E929eC4b771B73e874777cd9526546bEe3bBe',
       '0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270',
       '0xAa9654BECca45B5BDFA5ac646c939C62b527D394', 16944049, 1626488717],
      dtype=object)